In [ ]:
# !pip install sentence_transformers
# !pip install faiss-cpu

In [ ]:
import pandas as pd
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss 
import re

file_path = "DiseaseAndSymptoms.csv"
df1 = pd.read_csv(file_path)
df1.head(3)

e:\Superior University Semester 4\Programming for AI Lab\Assignment\12 Chatbot\virtual_enviroment\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df1 = df1.iloc[:, :4]  
print(df1.head())

In [ ]:
df1.sample()

In [ ]:
df1.count().sum()

In [ ]:
df1.isnull().sum()

In [ ]:
df1.to_csv("refine_data.csv", index=False)

In [ ]:
file_path = "refine_data.csv"
df2 = pd.read_csv(file_path)

In [ ]:
df2["Combined_Symptoms"] = df2.iloc[:, 1:].apply(lambda x: " ".join(x.dropna()), axis=1)
print(df2[["Disease", "Combined_Symptoms"]].head())  

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(df2["Combined_Symptoms"].values)

In [ ]:
print(embeddings.shape)

In [ ]:
import numpy as np
embeddings = np.array(embeddings)


In [ ]:
np.save('disease_sumptoms_embeddings.npy',embeddings)


In [ ]:

embeddings = np.load('disease_sumptoms_embeddings.npy')

In [ ]:
dimensions = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimensions)

In [ ]:
faiss_index.add(embeddings)

In [ ]:
faiss.write_index(faiss_index, 'faiss_index.faiss')

In [ ]:
def get_similar_movies(query,model,faiss_index,count=5):
    query_embeddings = model.encode([query])
    distance, indices = faiss_index.search(query_embeddings,count)

    for i in range(count):
        print(f"Disease {i+1}")
        print(f"Distance: {distance[0][1]}")
        print(df2["Combined_Symptoms"].iloc[indices[0][1]])

In [ ]:
get_similar_movies("disease",model,faiss_index,2)